# Processing MITGCM data for Scale Analysis 

**Purpose**: Code for producing data for temporal decorrelation scale analysis. Here, we will generate time series of temperature, salinity, and velocity profiles at the locations of the CCE1, CCE2, and CCE3 moorings. 

**Luke Colosi | lcolosi@ucsd.edu**

Force matplotlib plots to display directly within the output cell of the notebook: 

In [1]:
%matplotlib inline

Import python libraries

In [5]:
import sys
import numpy as np
import xarray as xr
from xmitgcm import open_mdsdataset

Set data analysis parameters

In [3]:
# Model parameters 
delta_t = 150  # Time steps of the raw model run (by raw, I mean the time increments that the model is ran at, not time increments that the diagnostics are output at). Units: seconds

# Set time and space parameters  
lat_cce  = [33.457, 34.3075, 34.44825228022894]                  # Specifies the latitude bounds for the region to analyze
lon_cce  = [-122.52233, -120.8042, -120.53825701527784]          # Specifies the longitude bounds for the region to analyze
encoding  = {'time': {'units': 'seconds since 2015-12-01 2:00'}} # Specifies the start time of the model run

# Set path to project directory
PATH_GRID   = '/data/SO2/SWOT/GRID/BIN/'                    # Space and time grid of the model 
PATH_OUTPUT = '/data/SO2/SWOT/MARA/RUN4_LY/DIAGS_HRLY/'     # Diagnostics of the model
PATH_nc     = '/data/SO3/lcolosi/mitgcm/SWOT_MARA_RUN4_LY/' # Directory to save netCDFs 
file_dim    = '3D'                                          # Set the dimension of the data (to include the depth or not where 3D is T,S,drhodr,vel and 2D is etan)

Load the grid and diagnostics data into a python structure. The diagnostics that we will be looking at include: 

1. **Potential Temperature** $\theta$: $^\circ C$
2. **Salinity** $S$: $g/kg$
3. **Stratification** $\frac{d\sigma}{dz}$: $kg/m^4$
4. **Zonal, meridional, and vertical velocity components**  $\textbf{u} = (u,v,w)$: $m/s$  

In [6]:
# Create dataset 
ds = open_mdsdataset(
    PATH_OUTPUT,                    # File path where the model output data is stored (.data and .meta files)
    PATH_GRID,                      # File path to the grid data of the model 
    iters='all',                    # Specifies which iteration of the data to load
    delta_t=delta_t, 
    ignore_unknown_vars=False,      # Specifies whether to ignore any unknown variables that may appear in the dataset
    prefix=['diags_' + file_dim],   # List of prefixes to filter the variables in the dataset
    ref_date="2015-01-01 02:00:00", # Specifies the starting point of the simulation time (which may include the spin up time before diagnostics are output)
    geometry='sphericalpolar'       # Specifies the  grid's geometry is spherical-polar. 
)

# Convert all variables and coordinates in the dataset to little-endian (the format how multi-byte values are stored into memory)

#--- Variables ---#
for var in ds.data_vars:
    if ds[var].dtype.byteorder == '>' or (ds[var].dtype.byteorder == '=' and sys.byteorder == "big"):  # Check if big-endian
        ds[var] = ds[var].astype(ds[var].dtype.newbyteorder('<'))

#--- Coordinates ---# 
for coord in ds.coords:
    if ds[coord].dtype.byteorder == '>'or (ds[coord].dtype.byteorder == '=' and sys.byteorder == "big"):  # Check if big-endian
        ds[coord] = ds[coord].astype(ds[coord].dtype.newbyteorder('<'))

Slice array based on longitude and latitude of CCE moorings

In [ ]:
# Get 2D coordinate fields
lat_YC = ds['YC']
lon_XC = ds['XC']
lat_YG = ds['YG']
lon_XG = ds['XG']

# Set dictionary to hold extracted profiles per variable
all_profiles = {var: [] for var in ds.data_vars}

# Loop through CCE sites
for i, (lat_target, lon_target) in enumerate(zip(lat_cce, lon_cce)):
    
    # --- For center-point variables (XC/YC) ---
    dist_sq_center = (lat_YC - lat_target)**2 + (lon_XC - lon_target)**2
    j_YC, i_XC = np.unravel_index(np.argmin(dist_sq_center.values), dist_sq_center.shape)
    
    # --- For U-point variables (XG/YC) ---
    dist_sq_u = (lat_YC - lat_target)**2 + (lon_XG - lon_target)**2
    j_YC_u, i_XG = np.unravel_index(np.argmin(dist_sq_u.values), dist_sq_u.shape)
    
    # --- For V-point variables (XC/YG) ---
    dist_sq_v = (lat_YG - lat_target)**2 + (lon_XC - lon_target)**2
    j_YG, i_XC_v = np.unravel_index(np.argmin(dist_sq_v.values), dist_sq_v.shape)
    
    # --- Loop through all variables and select appropriate index ---
    for var in ds.data_vars:
        da = ds[var]

        if {'YC', 'XC'}.issubset(da.dims):
            sel = da.isel(YC=j_YC, XC=i_XC)
        elif {'YC', 'XG'}.issubset(da.dims):
            sel = da.isel(YC=j_YC_u, XG=i_XG)
        elif {'YG', 'XC'}.issubset(da.dims):
            sel = da.isel(YG=j_YG, XC=i_XC_v)
        else:
            # For unexpected variable shapes
            print(f"Skipping {var}: unknown coordinate configuration.")
            continue
        
        # Add site dimension
        sel = sel.expand_dims(site=[f"CCE{i+1}"])
        all_profiles[var].append(sel)

# --- Combine all variables into datasets per variable ---
profiles_ds = xr.Dataset({var: xr.concat(all_profiles[var], dim='site') for var in all_profiles})

Save data into a netcdf file

In [ ]:
# Loop through each variable in the profiles dataset
for var in profiles_ds.data_vars:
    
    # Select the data array corresponding to the current variable
    da = profiles_ds[var]
    
    # Rechunk the data for faster writing:
    # Here we chunk along the 'time' dimension (adjust the chunk size as needed)
    da = da.chunk({'time': 1000})
    
    # Print status to monitor progress
    print(f"Saving {var}...")

    # Load the data into memory before saving (forces Dask to compute first),
    # which can speed up the actual file writing and reduce errors.
    da = da.load()
    
    # Save the data array to a NetCDF file
    da.to_netcdf(
        f"{PATH_nc}{var}_CCS4_hrly_ts_profile.nc",  # Output file name
        engine='scipy',                             # Use the faster 'scipy' NetCDF writer
        format='NETCDF3_64BIT',                     # Use a simpler NetCDF format (faster & widely compatible)
        encoding=encoding
    )


Saving THETA...
Saving SALT...
